In [11]:
from scripts import zoom_api
import pandas as pd

### Get Zoom Access Credentials

**Warning**: Don't print the token!

In [37]:
creds = zoom_api.Credentials.from_environment_variables()
oauth = zoom_api.get_server2server_oauth(
    account_id=creds.zoom_account_id,
    client_id=creds.zoom_client_id,
    client_secret=creds.zoom_client_secret,
)
token = oauth['access_token']
assert token

### Get Owner ID

Don't print it!

In [38]:
owner_id = pd.DataFrame(zoom_api.list_users(token=token)).query('role_id == "0"').iloc[0].id
assert owner_id

### Get Past Meetings

In [39]:
from datetime import datetime, timedelta

In [40]:
today = datetime.today()
two_months_ago = today - timedelta(days=30)
today, two_months_ago

In [41]:
two_months_ago.strftime("%Y-%m-%d")

In [42]:
today.strftime("%Y-%m-%d")

In [44]:
meeting_resp = zoom_api.list_past_meeting_sessions(
    token=token, 
    user_id=owner_id, 
    from_=two_months_ago.strftime("%Y-%m-%d"), 
    to=today.strftime("%Y-%m-%d"),
)
len(meeting_resp)

### Export Each Meeting's Info

In [65]:
def meeting_basename(meeting: dict) -> str:
    "Returns a basename for a file describing a meeting"
    import dateutil
    template = "meeting_{date}_{topic}_{id}"
    name = template.format(
        date=dateutil.parser.parse(meeting['start_time']).strftime("%Y-%m-%d"),
        topic=meeting['topic'].replace(' ', '-'),
        id=meeting['id'],
    )
    return name


meeting_basename(meeting_resp[0])

In [66]:
from pathlib import Path
import json

for meeting in meeting_resp:
    filename = Path(f"data/meetings/{meeting_basename(meeting)}.json")
    filename.parent.mkdir(parents=True, exist_ok=True)
    if not filename.exists():
        filename.write_text(json.dumps(meeting))
        


## Complete the Step 

### Export the Confirmation Flag File

In [ ]:
template = "data/access_dates/zoom_meetings_downloaded_%Y-%m-%d.flag"
filename = Path(today.strftime(template))
filename.touch(exist_ok=True)
assert filename.exists()
filename